In [1]:
import numpy as np
import os
from pynwb import NWBFile, NWBHDF5IO, image
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, ImagingPlane
from pynwb.file import Subject
from pynwb.device import Device
import math

import statistics
import pandas as pd
import h5py #Provides methods to supplement NWB
import datajoint as dj #
import csv 
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO
from matplotlib import pylab as plt

def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)
schema

#Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
schema = dj.schema('MarikeReimer', locals())
schema


#Define Mouse table
@schema
class Mouse(dj.Manual):
    definition = """
    subject_id: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    genotype: enum('B6', 'BalbC', 'Unknown', 'Thy1-YFP')
    sex: enum('M', 'F', 'Unknown')
    species: varchar(128)
    strain: varchar(128)
    """

@schema
class Session(dj.Manual):
    definition = """
    ->Mouse
    identifier: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    surgery: varchar(128)
    pharmacology: varchar(128)
    """


@schema
class Dendrite(dj.Manual):
    definition = """
    ->Session
    dendrite_id: int                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    soma_center_point: longblob
    proximal_dendrite_length: float
    medial_dendrite_length: float
    distal_dendrite_length: float
    """

@schema
class Image_segmentation(dj.Imported):
    definition = """
    ->Dendrite
    segmentation_name: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    length: float
    volume: float
    surface_area:float
    spine_type: enum('mushroom', 'thin', 'disconnected','U')
    center_of_mass: longblob
    """
    def make(self, key):
        subject_id = key['subject_id']
        identifier = key['identifier']

        nwbfile_to_read = 'C:/Users/meowm/OneDrive/TanLab/DataJointTesting/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb' #TODO: Remove hard coding
        #nwbfile_to_read = 'C:/Users/meowm/Downloads/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb'

        print(nwbfile_to_read)
        with NWBHDF5IO(nwbfile_to_read, 'r') as io:
            nwbfile = io.read()     
            for group in nwbfile.processing["SpineData"]["ImageSegmentation"].children[:]:
                #print(group.name)
                if group.name.startswith("Mushroom"):
                    spine_type = 'mushroom'
                elif group.name.startswith("Thin"):
                    spine_type = 'thin'
                elif group.name.startswith("Disconnected"):
                    spine_type = 'disconnected'
                else:
                    spine_type = 'U'

                length = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].length.data[:]
                length = length[0]
                volume = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].volume.data[:]
                volume = volume[0]
                surface_area = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].surface_area.data[:]
                surface_area = surface_area[0]
                center_of_mass = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].center_of_mass.data[:]
                center_of_mass = center_of_mass[0]
                
                key['segmentation_name'] = group.name 
                key['length'] = length
                key['volume'] = volume
                key['surface_area'] = surface_area
                key['spine_type'] = spine_type
                key['center_of_mass'] = center_of_mass
                self.insert1(key)

@schema
class Distance_to_soma(dj.Computed):
    definition = """
    ->Image_segmentation
    ---
    distance_to_soma: float"""
    def make(self, key):
        center_of_mass = (Image_segmentation() & key).fetch1('center_of_mass')
        soma_center_point = (Dendrite() & key).fetch1('soma_center_point')
        distance_to_soma = math.dist(center_of_mass,soma_center_point)

        key['distance_to_soma'] = distance_to_soma
        self.insert1(key)

#Instantiate tables
mouse = Mouse()
session = Session()
dendrite = Dendrite()
image_segmentation = Image_segmentation()
distance_to_soma = Distance_to_soma()

print(distance_to_soma)

[2023-08-08 09:49:22,755][INFO]: Connecting MarikeReimer@spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com:3306
[2023-08-08 09:49:23,188][INFO]: Connected MarikeReimer@spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com:3306


Schema `MarikeReimer`

*subject_id    *identifier    *dendrite_id   *segmentation_ distance_to_so
+------------+ +------------+ +------------+ +------------+ +------------+
L684           Dendrite1a     11             DisconnectedMu 47.4674       
L684           Dendrite1a     11             StubbySpine_0_ 17.7509       
L684           Dendrite1a     11             StubbySpine_10 104.197       
L684           Dendrite1a     11             StubbySpine_11 75.9831       
L684           Dendrite1a     11             StubbySpine_12 75.3099       
L684           Dendrite1a     11             StubbySpine_15 89.2687       
L684           Dendrite1a     11             StubbySpine_18 105.312       
L684           Dendrite1a     11             StubbySpine_1_ 29.4164       
L684           Dendrite1a     11             StubbySpine_2_ 54.7993       
L684           Dendrite1a     11             StubbySpine_3_ 63.3258       
L684           Dendrite1a     11             StubbySpine_4_ 68.2698       
L6

In [ ]:
mouse.drop()

In [47]:
mouse*session

subject_id Primary keys above the '---',identifier Primary keys above the '---',genotype,sex,species,strain,surgery,pharmacology
L684,Dendrite2a,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L684,Dendrite2b,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L684,Dendrite4b,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L692,Dendrite2,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L692,Dendrite5,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L698,Dendrite2a,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI
L698,Dendrite2b,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI
L711,Dendrite3,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L712,Dendrite1a,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L712,Dendrite1b,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI


In [23]:
session & 'subject_id = "L734"'

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology
L734,Dendrite1,Contusion,Vehicle


In [13]:
(session & 'subject_id = "L684"'& 'identifier = "Dendrite1b"').delete()

[2023-08-09 14:36:03,627][INFO]: Deleting 1 rows from `MarikeReimer`.`dendrite`
[2023-08-09 14:36:03,838][INFO]: Deleting 1 rows from `MarikeReimer`.`session`


Deletes committed.


1

In [89]:
session & 'subject_id = "L698"'

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology
L698,Dendrite2a,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI
L698,Dendrite2b,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI


In [86]:
(distance_to_soma & 'subject_id = "L698"' & 'identifier = "Dendrite1c"')

subject_id Primary keys above the '---',identifier Primary keys above the '---',dendrite_id Primary keys above the '---',segmentation_name Primary keys above the '---',distance_to_soma


In [ ]:
frame = pd.DataFrame(subject)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/SubjList.csv")

In [ ]:
frame = pd.DataFrame(subject*session*dendrite*image_segmentation*distance_to_soma)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/OP-09.csv")

In [ ]:
def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)

In [ ]:
Mouse.drop()